In [124]:
# libraries
from dotenv import load_dotenv
import os

# load environment variables from .env file
_ = load_dotenv()

In [125]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults

In [126]:
tool = TavilySearchResults(max_results=2)

In [127]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

对于`from langgraph.checkpoint.sqlite import SqliteSaver`的使用langchain课程进行了一个更新

In [128]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [129]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openollama)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)    # 这里添加checkpoint
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openollama(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        import threading
        print(f"take_action called in thread: {threading.current_thread().name}")
        tool_calls = state['messages'][-1].tool_calls
        results = []
        print(f"take_action called with tool_calls: {tool_calls}")
        for t in tool_calls:
            print(f"Calling: {t}") 
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                print(f"action {t['name']}, result: {result}")
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

关心的两件事情：
- `stream messages`: 这将决定采取什么行动的AI消息，以及代表行动结果的observation消息
- `token`: 对于llm调用的每一个token，我们希望能流式传输输出。

引入线程配置的概念：这用于跟踪持久性检查点内部的不同线程，这将允许我们同时进行多个对话。

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information.  \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOllama(
    model="qwen2.5:14b",
    temperature=0
)

query = "What is the weather in New York?"
messages = [HumanMessage(content=query)]
thread = {"configurable": {"thread_id": "1"}}  

with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])
    
    print ("-"*50)

    query = "What about in shanghai?"
    messages = [HumanMessage(content=query)]
    thread = {"configurable": {"thread_id": "1"}}   
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)

    print ("-"*50)

    query = "Which one is warmer?"
    messages = [HumanMessage(content=query)]
    thread = {"configurable": {"thread_id": "2"}}  
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    


[AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-12T15:58:55.510604Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5066746541, 'load_duration': 565119000, 'prompt_eval_count': 245, 'prompt_eval_duration': 3206000000, 'eval_count': 27, 'eval_duration': 1060000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-eb6eec2e-7a34-4408-b930-91ff8e760a5c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in New York'}, 'id': '3f15ea65-d0c5-4ddd-8606-84426c3ce8e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 245, 'output_tokens': 27, 'total_tokens': 272})]
take_action called in thread: MainThread
take_action called with tool_calls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in New York'}, 'id': '3f15ea65-d0c5-4ddd-8606-84426c3ce8e0', 'type': 'tool_call'}]
Calling: {'name': 'tavily_search_results_json', 'args': {'query'

`from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver`已经失效了，要使用`from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver`

In [135]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

query = "What is the weather in New York?"
messages = [HumanMessage(content=query)]
model = ChatOllama(
    model="llama3.2",
    temperature=0
)

# These events represent updates from the underlying stream.
async with AsyncSqliteSaver.from_conn_string("checkpoints.sqlite") as saver:
    abot = Agent(model, [tool], system=prompt, checkpointer=saver)
    config = {"configurable": {"thread_id": "thread-1"}}
    async for event in abot.graph.astream_events({"messages": messages}, config):
        kind = event["event"]
        # What we gonna to do is to find events that correspond to new tokens.
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # empty content in the context of Ollama means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content.
                print(content, end="|")

take_action called in thread: asyncio_0
take_action called with tool_calls: [{'name': 'tavily_search_results_json', 'args': {'query': 'New York City weather'}, 'id': 'e2a2d086-40f9-4000-9b77-df166ab861d2', 'type': 'tool_call'}]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'New York City weather'}, 'id': 'e2a2d086-40f9-4000-9b77-df166ab861d2', 'type': 'tool_call'}
action tavily_search_results_json, result: [{'title': 'Weather in New York City', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.714, 'lon': -74.006, 'tz_id': 'America/New_York', 'localtime_epoch': 1741795214, 'localtime': '2025-03-12 12:00'}, 'current': {'last_updated_epoch': 1741794300, 'last_updated': '2025-03-12 11:45', 'temp_c': 8.3, 'temp_f': 46.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14